## Author : Joseph Hencil Peter
### (Temasek Admission Id : 1880282G)
### Program/Code : RapidModelGenerator

#### Description: Auto generate the models in python language based on the minimum input given in the input text file. Each file will have code to record the accuracy, loss and execution time.

In [1]:
#import the required libraries
import os
import datetime as dt

import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, Activation
from keras.layers import Dense, Softmax
from keras.optimizers import SGD, RMSprop




Using TensorFlow backend.


In [56]:
#initialize variables (including hyper-parameters)
epochs=''
batch_size=''
layers = ''
num_classes ='10'
outFileName = ''
OptimizerUsed =''
Augmentation = ''

In [57]:
#read the models' input
modelInputFileName = 'InputModel.mi'


fileStat = os.stat(modelInputFileName)
if(fileStat.st_size == 0):
    print("Model Input file is empty or doesn't exist. Please check...")
    assert(False)

#output folder
outFolder = 'SampleOut_' + str(dt.datetime.now()) 
outFolder = outFolder.replace(':','-') 
outFolder = outFolder.replace(' ','_')
formattedOutFolder = '.\\' + outFolder + '\\'
googleBaseFolder = '/content/gdrive/My Drive/CNNProject/' + outFolder + '/'
print(outFolder)
print(formattedOutFolder)
print(googleBaseFolder)


SampleOut_2018-12-03_23-22-30.624946
.\SampleOut_2018-12-03_23-22-30.624946\
/content/gdrive/My Drive/CNNProject/SampleOut_2018-12-03_23-22-30.624946/


In [58]:
def GetImportSection():
    importSection="import keras\nfrom keras import callbacks\nfrom keras.datasets import cifar10\nfrom keras.models import Sequential\n"
    
    importSection += "from keras.layers import Dense, Activation, Softmax, Dropout\nfrom keras.layers import Conv2D, MaxPooling2D, Flatten\n"
    importSection += "from keras.optimizers import SGD, RMSprop\nfrom keras import backend as K\n"

    importSection += "import numpy as np"
    importSection += "\nfrom datetime import datetime as dt" 
    importSection += "\nfrom keras.preprocessing.image import ImageDataGenerator"

    #import numpy as np\
    #import matplotlib.pyplot as plt\
    #%matplotlib inline\
    #%config InlineBackend.figure_format='retina'\
    #plt.style.use(\'ggplot\')'

    return importSection

In [59]:
def GetDataLoadAndPreprocessingSection():
    dataLoadAndPreProcess = ''
    dataLoadAndPreProcess +="\n(x_train, y_train), (x_test, y_test) = cifar10.load_data()"
    dataLoadAndPreProcess +="\nimg_rows, img_cols = 32, 32"
    dataLoadAndPreProcess +="\nx_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)"
    dataLoadAndPreProcess +="\nx_test = x_test.reshape(x_test.shape[0],  img_rows, img_cols, 3)"
    dataLoadAndPreProcess +="\nx_train = x_train.astype('float32')"
    dataLoadAndPreProcess +="\nx_test = x_test.astype('float32')"
    dataLoadAndPreProcess +="\nx_train /= 255"
    dataLoadAndPreProcess +="\nx_test /= 255"
    dataLoadAndPreProcess +="\ny_train = keras.utils.to_categorical(y_train, num_classes)"
    dataLoadAndPreProcess +="\ny_test = keras.utils.to_categorical(y_test, num_classes)"
    
    return dataLoadAndPreProcess

In [60]:
def GetTrainingSection():
   
    trainingSection = 'model.compile(loss=keras.losses.categorical_crossentropy,optimizer=' + OptimizerUsed + ',metrics=[\'accuracy\'])'
    
    #trainingSection += '\nmodel_checkpoints = callbacks.ModelCheckpoint(\'' + outFolder + '\\' + outFileName + '_weights_{epoch:02d}_{val_loss:.2f}_Proj.h5\', monitor=\'val_loss\','
    trainingSection += '\nmodel_checkpoints = callbacks.ModelCheckpoint(\'' + googleBaseFolder  + outFileName + '_weights_{epoch:02d}_{val_loss:.2f}_Proj.h5\', monitor=\'val_loss\','
    
    trainingSection += 'verbose=1, save_best_only=True, save_weights_only=False, mode=\'auto\', period=1)'
    
    if (Augmentation == 'True'):
        trainingSection += '\n\n#Data Augmentation Enabled'
        trainingSection += '\ndatagen = ImageDataGenerator( rotation_range=90, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)' 
        trainingSection += '\ndatagen.fit(x_train)'
        trainingSection += '\nmodel.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,steps_per_epoch=32,'
        trainingSection += '\nvalidation_data=(x_test, y_test),'
        trainingSection += '\nworkers=4)'
    else:
        trainingSection += '\n\n#Training'
        trainingSection += '\nmodel_log = model.fit(x_train, y_train,'
        trainingSection += 'batch_size=batch_size, epochs=epochs, '
        trainingSection += 'validation_data=(x_test, y_test), callbacks=[model_checkpoints])'
    
    return trainingSection

In [61]:
def GetEvaluationSection():
    evaluation='score = model.evaluate(x_test, y_test, verbose=0)\n'
    evaluation+='\nprint(\'Test loss:\', score[0])'
    evaluation+='\nprint(\'Test accuracy:\', score[1])'
    evaluation+='\nfileEvaluation = open(\'' + googleBaseFolder +  'EvaluationReport.txt\', \'a+\')'
    #evaluation+='\nfileEvaluation.write(\'File: ' + outFileName + ' Accuracy : ' + str(score[1])+ 'loss : ' + str(score[0]) + 'Training Time(S) : ' + trainingTime))'
    evaluation+='\nfileEvaluation.write(\'\\nFile: ' + outFileName + '\\tAccuracy : \' + str(score[1]) + \'\\tLoss : \' + str(score[0]) + \'\\tTraining Time(S) : \' + trainingTime + \'\')'
    evaluation+='\nfileEvaluation.close()'
    return evaluation

In [62]:
def GetFileHeader():
    headerSection = "#*****************************************"
    headerSection += "\n#Author : Hencil Peter"
    headerSection += "\n#File Name : " + outFileName
    headerSection += "\n#Timestamp : " + str(dt.datetime.now())
    headerSection += "\n#*****************************************"
    return headerSection

In [63]:


def ProcessHyperparameters(token):
    global batch_size
    global epochs
    
    collenIndex = token.index(':')
    key = token[:collenIndex]
    value = token[collenIndex + 1 :]
    if (key=='BatchSize'):
        batch_size=value
    elif(key=='Epochs'):
        epochs = value
        
    #print('batch size =', batch_size, 'epochs =', epochs)

In [64]:
def ProcessHeader(token):
    global outFileName
    collenIndex = token.index(':')
    outFileName = token[collenIndex + 1 :] + ".py"

In [65]:
def ProcessOptimizer(token):
    global OptimizerUsed
    collenIndex = token.index(':')
    key = token[:collenIndex]
    value = token[collenIndex + 1 :]
    value = value.strip()
    print('Key', key)
    print('Value', value)
    if (value=='SGD'):
        OptimizerUsed = 'SGD(lr=0.01)'
    elif(value=='RMS'):
        OptimizerUsed = 'RMSprop(lr=0.0001, decay=1e-6)'

In [66]:
def ProcessAugmentation(token):
    global Augmentation
    collenIndex = token.index(':')
    key = token[:collenIndex]
    value = token[collenIndex + 1 :]
    value = value.strip()
    if (value=='True'):
        Augmentation = 'True'
    else:
        Augmentation = 'False'
        
    
    

In [67]:
def GetNextToken(input):
    token = ''
    tokenExcludedString = ''
    
    if (input.find(';')!= -1):
        collenIndex = input.index(';')
        token = input[:collenIndex]
        tokenExcludedString = input[collenIndex + 1:]
    elif len(input) > 0 :
        token = input
        
    return token, tokenExcludedString

In [68]:
def GetNextParameter(input):
    if (len(input) == 0):
        return '',''
    
    parameter = ''
    parameterExcludedString = ''
    
    if (input.find(',')!= -1):
        commaIndex = input.index(',')
        parameter = input[:commaIndex]
        parameterExcludedString = input[commaIndex + 1:]
    elif len(input) > 0 :
        parameter = input
        
    return parameter, parameterExcludedString

In [69]:
def GetParameter(parametersList, prefix):
    parameter = ''
    
    for index, value in enumerate(parametersList):
        if value.startswith(prefix):
            parameter = value
            
    return parameter
    

In [70]:
def AddModel(model, token):
    collenIndex = token.index(':')
    modelName = token[collenIndex + 1:]
    #print(modelName)
    if (modelName == 'Sequential'):
        model = Sequential  
    str='model = Sequential()'
    global layers
    layers += '\n'+ str

In [71]:
def AddConvoltionLayer(model, token):
    collenIndex = token.index(':')
    
    parameters = token[collenIndex + 1:]
     
    filter =''
    kernel = ''
    padding =''
    imageSize =''
    parameterList = parameters.split(',')
    
    #print(parameterList)
    if (len(parameterList) > 0):
        filter = GetParameter(parameterList, 'Filter')
        kernel = GetParameter(parameterList, 'Kernel')
        padding = GetParameter(parameterList, 'Padding')
        imageSize = GetParameter(parameterList, 'InputSize')
        bias = GetParameter(parameterList, 'Bias')
        kInit = GetParameter(parameterList, 'KInit')
        bInit = GetParameter(parameterList, 'BInit')
    
 
    
    str = 'model.add(Conv2D('
    if (len(filter) > 0): #Add Filter
        filter = filter[filter.index('|') + 1:]
        str += filter
    
    if (len(kernel)>0): #Add Kernel
        kernel = kernel[kernel.index('|') + 1:]
        underscoreIndex = kernel.index('_')
        str += ',kernel_size=(' + kernel[ : underscoreIndex] + ',' + kernel[ underscoreIndex + 1:] + ')'
        
    
    if (len(padding) > 0):#Add Padding
        padding = padding[padding.index('|') + 1:]
        str+= ",padding='" + padding + "'"
    
    if (len(imageSize) > 0):#Add Input Shape 
        imageSize = imageSize[imageSize.index('|') + 1:]
        parts = imageSize.split('_')
        str += ',input_shape=(' +  parts[0] + ',' + parts[1] + ',' + parts[2] +')'
        
    if (len(bias)>0): # Add Bias
        bias = bias[bias.index('|') + 1:]
        str += ",use_bias='" + bias + "'"
    
    if (len(kInit) > 0): #Add Kernel Initializer
        kInit = kInit[kInit.index('|') + 1:]
        str += ",kernel_initializer='" + kInit + "'"
    
    if (len(bInit) > 0): # Add Bias Initializer
        bInit = bInit[bInit.index('|') + 1:]
        str += ",bias_initializer='" + bInit + "'"
                    
    str+='))'
    #print(str)
    global layers
    layers += '\n' + str

In [72]:
def AddActivationLayer(model, token):
    
    collenIndex = token.index(':')
    
    activationFunctionName = token[collenIndex + 1:]
    
    str = "model.add(Activation(\'"  + activationFunctionName.strip() + "\'))"
    #print(str)
    global layers
    layers += '\n' + str

In [73]:
 def AddMaxPooling2DLayer(model, token):
    collenIndex = token.index(':')
    
    poolSize = token[collenIndex + 1:]
    underscoreIndex =  poolSize.index('_')        
    str = "model.add(MaxPooling2D(pool_size=("  + poolSize[ : underscoreIndex] + ',' + poolSize[ underscoreIndex + 1:]  + ")))"
    #print(str)
    global layers
    layers += '\n' + str 

In [74]:
def AddDropoutLayer(model, token):
    collenIndex = token.index(':')
    
    dropout = token[collenIndex + 1:]
    str = "model.add(Dropout(" + dropout  + "))"
    #print(str)
    global layers
    layers += '\n'+ str

In [75]:
def AddDenseLayer(model, token):
    collenIndex = token.index(':')
    
    dense = token[collenIndex + 1:]
    str = "model.add(Dense(" + dense  + "))"
    #print(str)
    global layers
    layers += '\n' + str

In [76]:
def AddFlattenLayer(model, token):
    str = "model.add(Flatten())"
    #print(str)
    global layers
    layers += '\n' + str

In [77]:
def ProcessToken(model, token):
    if (len(token) == 0):
        return

    #print('Token : ', token)
    

    
    if (token.startswith('Type')): # Type of the model
        AddModel(model, token)
    elif (token.startswith('Conv2D')):
        AddConvoltionLayer(model, token)
    elif (token.startswith('Activation')):
        AddActivationLayer(model, token)
    elif (token.startswith('MaxPooling2D')):
        AddMaxPooling2DLayer(model, token)
    elif (token.startswith('Dropout')):
        AddDropoutLayer(model, token)
    elif (token.startswith('Dense')):
        AddDenseLayer(model, token)
    elif (token.startswith('Flatten')):
        AddFlattenLayer(model, token)
    elif(token.startswith('BatchSize') | token.startswith('Epochs')):
        ProcessHyperparameters(token)
    elif(token.startswith('Opt')):
        ProcessOptimizer(token)
    elif(token.startswith('Augmentation')):
        ProcessAugmentation(token)
    elif(token.startswith('Id')):
        ProcessHeader(token)
        
        
        
        

In [78]:
#Create and Save python file

def CreateAndSaveModelFile():
    print(formattedOutFolder)
    if not os.path.exists(formattedOutFolder):
        os.makedirs(formattedOutFolder)
    
    modelOutputFile = open(formattedOutFolder + outFileName,'w')
    modelOutputFile.write(GetFileHeader())
    modelOutputFile.write('\n\n')
    modelOutputFile.write(GetImportSection())
    modelOutputFile.write('\n\n#Hyper parameters')
    modelOutputFile.write('\nbatch_size=' + batch_size)
    modelOutputFile.write('\nepochs=' + epochs)
    modelOutputFile.write('\nnum_classes =' + num_classes )
    modelOutputFile.write('\ntrainingTime=\'\'');

    modelOutputFile.write('\n\n#Preprocess the data')
    modelOutputFile.write(GetDataLoadAndPreprocessingSection())
    modelOutputFile.write('\n\n#Construct network Layers')
    modelOutputFile.write(layers)
    modelOutputFile.write('\n\n#Train the model\n')
    modelOutputFile.write('\nt1 = dt.now()\n')
    modelOutputFile.write(GetTrainingSection())
    modelOutputFile.write('\nt2 = dt.now()')
    modelOutputFile.write('\ndelta = t2 - t1')
    modelOutputFile.write('\ntrainingTime = str(delta.total_seconds())')
    modelOutputFile.write('\n\n#Evaluate the accuracy\n')
    #modelOutputFile.write('print(\'Score = \',model_log[0],\'Loss = \', model_log[1])' )
    modelOutputFile.write(GetEvaluationSection())
    modelOutputFile.close()

In [79]:
modelInput = open(modelInputFileName,'r')
modelInputList = modelInput.readlines()
modelCount = 0
model = Sequential
global outFolder
#global OptimizerUsed
for input in modelInputList:
    modelCount+=1
    layers = ''
    #print('Started Constructing Model : ', modelCount, input)
    tokenExcludedString = input
    #print('Token : ', tokenExcludedString)
    while (len(tokenExcludedString) > 0 ):
        token, tokenExcludedString = GetNextToken(tokenExcludedString)
        ProcessToken(model, token)
        #print('token : ', token)
        
 
    #print('optimizer',OptimizerUsed)
    CreateAndSaveModelFile()
    print('Completed the current network layer construction..')
        
        

    



Key Opt
Value RMS
optimizer RMSprop(lr=0.0001, decay=1e-6)
.\SampleOut_2018-12-03_23-22-30.624946\
Completed the current network layer construction..
Key Opt
Value RMS
optimizer RMSprop(lr=0.0001, decay=1e-6)
.\SampleOut_2018-12-03_23-22-30.624946\
Completed the current network layer construction..
Key Opt
Value RMS
optimizer RMSprop(lr=0.0001, decay=1e-6)
.\SampleOut_2018-12-03_23-22-30.624946\
Completed the current network layer construction..
Key Opt
Value RMS
optimizer RMSprop(lr=0.0001, decay=1e-6)
.\SampleOut_2018-12-03_23-22-30.624946\
Completed the current network layer construction..
Key Opt
Value RMS
optimizer RMSprop(lr=0.0001, decay=1e-6)
.\SampleOut_2018-12-03_23-22-30.624946\
Completed the current network layer construction..
Key Opt
Value RMS
optimizer RMSprop(lr=0.0001, decay=1e-6)
.\SampleOut_2018-12-03_23-22-30.624946\
Completed the current network layer construction..
Key Opt
Value RMS
optimizer RMSprop(lr=0.0001, decay=1e-6)
.\SampleOut_2018-12-03_23-22-30.624946\
C

In [80]:
print('Models are Generated!!!!!!!!!!!!!')

Models are Generated!!!!!!!!!!!!!
